# This script shows an example of finding grocery stores from OpenStreetMap.

In [1]:
import geopandas as gpd
import os
import pandas as pd

from shapely import wkt
import sqlite3

# Add java and OSM to PATH

In my installation I have several *osmosis* and *java* applications for testing, and hence I have not added them to the path. Hence I will set the path here before running command-line arguments.

In [ ]:
print(os.environ["PATH"])

In [ ]:
os.environ["PATH"] = r"C:\Program Files\Microsoft\jdk-17.0.9.8-hotspot\bin" + os.pathsep + os.getenv("PATH") + os.pathsep + r"C:\MyPrograms\osmosis-0.49.2\bin"

In [ ]:
print(os.environ["PATH"])

# Now run *osmosis* to pull the points of interest from OSM

**Note that some points of interest (POIs) are marked as nodes and some as polygons (ways) in OSM. Hence we need to pull both.

The ! in Jupyter runs a command line program

In [ ]:
%cd C:\Users\bsharma2\AccessOpportunities\osm\20240118

In [ ]:
!osmosis --read-pbf peel-york-durham.osm.pbf --nkv keyValueList="shop.supermarket","shop.grocery","shop.food","shop.greengrocer" --wb grocery_nodes.osm.pbf 

In [ ]:
!osmosis --read-pbf peel-york-durham.osm.pbf --wkv keyValueList="shop.supermarket","shop.grocery","shop.food","shop.greengrocer" --used-node --wb grocery_ways.osm.pbf 

# Now process the output

## Version 1 -- export to shapefiles

When reading OSM files into QGIS, it only reads the first 100 entries. Hence if we want to do this then we need to export to a shapefile first. We can do this using the `ogr2ogr`, which fortunately is included in the r5py environment that we're using here.

In [ ]:
# !ogr2ogr -f "ESRI Shapefile" grocery_nodes.shp grocery_nodes.osm.pbf points

In [ ]:
# !ogr2ogr -f "ESRI Shapefile" grocery_ways.shp grocery_ways.osm.pbf multipolygons

## Version 2 -- export to SQLite, which we'll read into geopandas

Alternately, we can read these files directly into pandas and geopandas.  Following the example from https://python.plainenglish.io/exploring-openstreetmap-data-using-geopandas-d62b55fc40a4, I will use a SQLite as the intermediate format.

Note that the table from an OSM export are are: 'geometry_columns', 'spatial_ref_sys'  'points', 'line', 'multilinestri', 'multipoly', 'other_relans'

We need the 'points' table for the node tags and the 'multipoly' table for the ways tags.

In [ ]:
!ogr2ogr -f SQLite -lco FORMAT=WKT grocery_nodes.sqlite grocery_nodes.osm.pbf

In [ ]:
!ogr2ogr -f SQLite -lco FORMAT=WKT grocery_ways.sqlite grocery_ways.osm.pbf

### Read in the nodes file from SQLite into (geo)pandas

Note that I'm only bringing select columns into the DataFrame

In [ ]:
df_nodes = pd.read_sql(f"SELECT ogc_fid, WKT_GEOMETRY, name FROM points;", sqlite3.connect("./grocery_nodes.sqlite"))

In [ ]:
df_nodes['geom'] = gpd.GeoSeries.from_wkt(df_nodes['WKT_GEOMETRY'])
df_nodes = df_nodes.drop('WKT_GEOMETRY', axis=1)
df_nodes = df_nodes.set_index('ogc_fid')
gdf_nodes = gpd.GeoDataFrame(df_nodes, geometry='geom')
print(gdf_nodes.shape)

### Read in the ways file from SQLite into (geo)pandas

In [ ]:
df_ways = pd.read_sql(f"SELECT ogc_fid, WKT_GEOMETRY, name FROM multipolygons;", sqlite3.connect("./grocery_ways.sqlite"))

In [ ]:
df_ways['geom'] = gpd.GeoSeries.from_wkt(df_ways['WKT_GEOMETRY'])
df_ways = df_ways.drop('WKT_GEOMETRY', axis=1)
df_ways = df_ways.set_index('ogc_fid')
gdf_ways = gpd.GeoDataFrame(df_ways, geometry='geom')
print(gdf_ways.shape)

### Combine the nodes and ways tags into a single DataFrame

Steps:
1. Find the representative point for each way polygon, and set this as the geometry
2. Combine the dataframes together. I only know how to do this using pandas commands, hence
3. Reset as GeoDataFrame

In [ ]:
gdf_ways['repr_pt'] = gdf_ways.geom.representative_point()
gdf_ways = gdf_ways.set_geometry('repr_pt')
gdf_ways = gdf_ways.drop('geom', axis=1)
gdf_ways = gdf_ways.rename({'repr_pt': 'geom'}, axis=1)

In [ ]:
gdf_ways.head()

In [ ]:
gdf_nodes.head()

In [ ]:
combined_df = pd.concat([gdf_nodes, gdf_ways], axis=0).reset_index()

In [ ]:
combined_gdf = gpd.GeoDataFrame(combined_df, geometry='geom')

In [ ]:
combined_gdf

## We're done, export to file

In [ ]:
combined_gdf.to_csv("osm_grocery_stores.csv")